# ⚙️ TensorFlow 2 Basics
**Source: TensorFlow in Action – Chapter 2: TensorFlow 2**[file:1]

Chapter 2 mengenalkan dasar eksekusi TensorFlow 2, mulai dari eager execution, tipe data utama (`tf.Variable`, `tf.Tensor`, `tf.Operation`), sampai operasi inti neural network seperti matrix multiplication, convolution, dan pooling.




In [2]:
import tensorflow as tf
import numpy as np


## ⚡ Eager Execution dan `tf.function`

**Theory**: TensorFlow 2 memakai eager execution (define-by-run), sehingga operasi langsung dieksekusi seperti NumPy tanpa `Session` dan `Graph` eksplisit.

`@tf.function` mengubah fungsi Python berisi operasi TF menjadi graph yang bisa dioptimasi, tanpa mengubah gaya penulisan imperatif.


In [3]:
# Eager execution aktif secara default di TensorFlow 2
x = tf.constant([[1., 2.], [3., 4.]])
y = tf.constant([[5., 6.], [7., 8.]])

z = tf.matmul(x, y)  # langsung dieksekusi
print("Eager matmul result:\n", z.numpy())

@tf.function
def matmul_fn(a, b):
    return tf.matmul(a, b)

z_graph = matmul_fn(x, y)
print("Graph matmul result (tf.function):\n", z_graph.numpy())


Eager matmul result:
 [[19. 22.]
 [43. 50.]]
Graph matmul result (tf.function):
 [[19. 22.]
 [43. 50.]]


## 🧱 Tensor, Variable, dan Operation

**Theory**: Tiga building block utama TF2 adalah:
- `tf.Tensor`: nilai immutable (hasil komputasi/konstanta)
- `tf.Variable`: nilai trainable yang dapat di-update selama training
- `tf.Operation`: node komputasi yang mentransformasikan tensor/variable

Sebagian besar model deep learning hanyalah komposisi dari tiga komponen ini.


In [4]:
# Tensor (immutable)
t = tf.constant([1.0, 2.0, 3.0])

# Variable (trainable / mutable)
v = tf.Variable([1.0, 2.0, 3.0])
v.assign_add([0.5, 0.5, 0.5])  # update in-place

print("Tensor:", t.numpy())
print("Variable after update:", v.numpy())


Tensor: [1. 2. 3.]
Variable after update: [1.5 2.5 3.5]


## ➕ Operasi Dasar dan Reduksi

**Theory**: Operasi reduksi seperti `reduce_sum` dan `reduce_mean` menyederhanakan
tensor dengan menjumlahkan atau merata-ratakan elemen di sepanjang *axis* tertentu.

Contoh: rata-rata baris tensor

$$
A \in \mathbb{R}^{2 \times 2}
$$

dituliskan sebagai

$$
\mu = \text{mean}(A,\ \text{axis}=0)
$$


In [5]:
a = tf.constant([[1., 2.], [3., 4.]])

sum_all = tf.reduce_sum(a)              # skalar
mean_axis0 = tf.reduce_mean(a, axis=0)  # rata-rata per kolom
mean_keepdims = tf.reduce_mean(a, axis=0, keepdims=True)

print("Tensor a:\n", a.numpy())
print("Sum all:", sum_all.numpy())
print("Mean axis=0:", mean_axis0.numpy())
print("Mean axis=0, keepdims=True:\n", mean_keepdims.numpy())


Tensor a:
 [[1. 2.]
 [3. 4.]]
Sum all: 10.0
Mean axis=0: [2. 3.]
Mean axis=0, keepdims=True:
 [[2. 3.]]


## 🧮 Matrix Multiplication

**Theory**: Perkalian matriks di TensorFlow menggunakan `tf.matmul`.

Jika

$$
x \in \mathbb{R}^{1 \times 2}
$$

dan

$$
W \in \mathbb{R}^{2 \times 2},
$$

maka output dihitung sebagai

$$
y = xW
$$



In [6]:
W = tf.constant([[1.0, -1.0], [0.5, 2.0]])   # 2x2
x_vec = tf.constant([[2.0, 3.0]])            # 1x2

y_vec = tf.matmul(x_vec, W)  # hasil: 1x2
print("xW =", y_vec.numpy())


xW = [[3.5 4. ]]


## 🧱 Convolution 2D

**Theory**: Pada operasi convolution 2D, sebuah kernel berukuran

$$
\mathbb{R}^{k_h \times k_w}
$$

digeser di atas input, kemudian dilakukan operasi perkalian dan penjumlahan untuk menghasilkan nilai output pada setiap posisi.

Secara matematis, proses convolution 2D dapat dituliskan sebagai:

$$
y[i, j] =
\sum_{u=0}^{k_h-1}
\sum_{v=0}^{k_w-1}
x[i+u, j+v] \cdot w[u, v]
$$

Dalam TensorFlow, citra grayscale 2D terlebih dahulu diubah ke dalam bentuk tensor 4D dengan format:

$$
[B, H, W, C]
$$

sebelum digunakan pada fungsi `tf.nn.convolution` atau `tf.nn.conv2d`, di mana `B` menyatakan ukuran batch, `H` dan `W` adalah tinggi dan lebar citra, serta `C` adalah jumlah kanal.


In [7]:
# Contoh kecil: citra 5x5 dan kernel 3x3 (edge detector sederhana)
img = tf.constant([
    [0., 0., 0., 0., 0.],
    [0., 1., 1., 1., 0.],
    [0., 1., 1., 1., 0.],
    [0., 1., 1., 1., 0.],
    [0., 0., 0., 0., 0.]
])

sobel = tf.constant([
    [-1., -2., -1.],
    [ 0.,  0.,  0.],
    [ 1.,  2.,  1.]
])

# reshape ke format [batch, h, w, c]
img4d = tf.reshape(img, [1, 5, 5, 1])
kernel4d = tf.reshape(sobel, [3, 3, 1, 1])

edge = tf.nn.convolution(img4d, kernel4d, padding="VALID")
print("Edge map (squeezed):\n", tf.squeeze(edge).numpy())


Edge map (squeezed):
 [[ 3.  4.  3.]
 [ 0.  0.  0.]
 [-3. -4. -3.]]


## 🪣 Max Pooling dan Average Pooling

**Theory**: Pada operasi max pooling 2D dengan ukuran jendela

$$
k \times k
$$

setiap nilai output dihitung sebagai nilai maksimum dari elemen-elemen input
yang berada di dalam jendela tersebut.

Secara matematis, max pooling dapat dituliskan sebagai:

$$
y[i, j] = \max_{(u, v) \in \text{window}} x[u, v]
$$

Pada average pooling, operasi maksimum digantikan dengan perhitungan rata-rata
dari seluruh elemen yang berada di dalam jendela.

Secara matematis, average pooling dirumuskan sebagai:

$$
y[i, j] = \frac{1}{k^2}
\sum_{(u, v) \in \text{window}} x[u, v]
$$


In [8]:
# gunakan output edge dari contoh sebelumnya
edge4d = tf.reshape(edge, [1, edge.shape[1], edge.shape[2], 1])

max_pooled = tf.nn.max_pool(
    edge4d,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding="VALID"
)

avg_pooled = tf.nn.avg_pool(
    edge4d,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding="VALID"
)

print("Max pooling result:\n", tf.squeeze(max_pooled).numpy())
print("Avg pooling result:\n", tf.squeeze(avg_pooled).numpy())


Max pooling result:
 4.0
Avg pooling result:
 1.75


## ✅ Penutup

**Theory**: Dengan memahami konsep *eager execution* serta objek dasar dalam TensorFlow, yaitu
Tensor, Variable, dan Operation, serta operasi inti pada neural network seperti
matrix multiplication (matmul), convolution, dan pooling, telah terbentuk
fondasi yang kuat untuk melanjutkan penggunaan API tingkat tinggi Keras
pada chapter selanjutnya.
